    Duncan Rocha
    3/22/18
    STELA Collective

    Example code for how one might use the server to send commands to the arduino.

In [2]:
import numpy as np
import json
import time
import urllib2

In [16]:

class server:
    def __init__(self):
        # Specify flask ip adress here (this one works if flask is run on your own computer)
        self.url = 'http://192.168.0.101:5123'
        
    def set_location(self,lon_lat,units):
        data = {'location': lon_lat,'units':units}
        req = urllib2.Request(self.url+'/set_location')
        req.add_header('Content-Type', 'application/json')
        response = urllib2.urlopen(req, json.dumps(data))
        
    def set_targ(self,az_alt):
        # Set the target position in the arduino (notice dictionary protocol)
        data = {'targ': az_alt}
        req = urllib2.Request(self.url+'/set_targ')
        req.add_header('Content-Type', 'application/json')
        response = urllib2.urlopen(req, json.dumps(data))
        
    def get_pos(self):
        # Get the current position of the arduino (again, protocols)
        a = urllib2.urlopen(self.url+'/get_pos')
        dic = json.loads(a.read())
        az_alt = dic['pos']
        #az_alt_targ = dic['targ']
        return az_alt#,az_alt_targ
    
    def setup(self):
        # Currently nonfuntional
        a = urllib2.urlopen(self.url+'/setup')
        return json.loads(a.read())

    def move(self,az_alt):
        # move a small incriment
        data = {'increment': az_alt}
        req = urllib2.Request(self.url+'/move')
        req.add_header('Content-Type', 'application/json')
        response = urllib2.urlopen(req, json.dumps(data))
        
    def set_calib(self,n=None):
        if n==None:
            urllib2.urlopen(self.url+'/set_calib')
        else:
            data = {"calib_num": n }
            req = urllib2.Request(self.url+'/set_calib')
            req.add_header('Content-Type', 'application/json')
            response = urllib2.urlopen(req, json.dumps(data))
    def calibrate(self):
        urllib2.urlopen(self.url+'/calibrate')

    
serv = server()
serv.set_location([-112,34],["deg","deg"])
coors = serv.setup()
#print coors


"""for i in range(100):
    serv.move([1,1])
    print serv.get_pos()
#serv.setup()
#serv.move([10,10])
#serv.move([10,10])
#print serv.get_pos()"""

'for i in range(100):\n    serv.move([1,1])\n    print serv.get_pos()\n#serv.setup()\n#serv.move([10,10])\n#serv.move([10,10])\n#print serv.get_pos()'

In [45]:
import time
spd = 10
last = time.time()
pos=0

while True:
    looptime = time.time()-last
    last = time.time()
    pos += spd*looptime
    print pos
    if pos > 360:
        pos -360
    serv.set_targ([pos,0])
    time.sleep(0.1)
    
    #print serv.get_pos()

0.001220703125
1.05533123016
2.16280221939
3.2205915451
4.2707824707
5.32848119736
6.39804124832
7.48779058456
8.59153270721
9.64298248291
10.7513713837
11.8023109436
12.9163408279
14.004149437
15.0604486465
16.1236071587
17.18313694
18.2765960693
19.3798160553
20.4421758652
21.4907455444
22.5949454308
23.6563634872
24.7480535507
25.7956528664
26.9067931175
28.0122423172
29.1208124161
30.1877307892
31.2432217598
32.3547029495
33.4480929375
34.5587134361
35.6053543091
36.6585350037
37.7224159241
38.8350963593
39.8895263672
40.9386968613
42.0224547386
43.1162953377
44.1703653336
45.2801465988
46.3859272003
47.4868869781
48.5650753975
49.6651053429
50.7218241692
51.7747044563
52.8807234764
53.9387226105
55.0122117996
56.1111307144
57.1994400024
58.254070282
59.3605494499
60.4068398476
61.4591503143
62.5719809532
63.6793208122
64.7808790207
65.8286499977
66.9474411011
67.9985404015
69.085021019
70.147061348
71.2034416199
72.3088812828
73.3571100235
74.4642996788
75.5316305161
76.638250351


KeyboardInterrupt: 

In [ ]:
last = Time.T

In [37]:
import astropy.coordinates as cp
import astropy.units as u
from astropy.time import Time

cel = cp.SkyCoord([u'101.287 6.75248', u'79.1723 5.27816', u'114.825 7.65503'],unit=[u.deg,u.deg])

lon=-115
lat=30

location = cp.EarthLocation(lon*u.deg,lat*u.deg)
altaz = cp.AltAz(location=location,obstime=Time.now())

obs = cel.transform_to(altaz)
obsl = np.array([obs.az.deg,obs.alt.deg]).T

print obsl

[[220.95097788  60.78757991]
 [245.55770843  44.25407654]
 [193.46312879  67.07432739]]


In [38]:

for i in [0,1,2]:
    serv.set_targ(obsl[i].tolist())
    time.sleep(5)
    serv.set_calib()
    
serv.calibrate()

In [30]:
serv.get_pos()

[273.82501, 38.58749]